In [122]:
import pandas as pd
import requests

In [123]:
queries = [ "http://localhost:8983/solr/imdb_movies/select?defType=dismax&fl=score%20*%20[features]&indent=true&q.op=OR&q=christmas%5E2%20santa%5E2%20snow%20elf%20rodolf%20festive%20claus%20merry%20holliday%20%22candy%20cane%22%20%22christmas%20tree%22%20%22christmas%20eve%22%20advent&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1&rows=300" ]

In [124]:
def extract_features(row):
    result = []
    features = row['[features]'].split(',')

    for idx, feature in enumerate(features, start=1):
        result.append(f" {idx}:{feature.split('=')[1]}")
    
    return ''.join(result)

def convert_to_dat(row, relevant):
    # <line> .=. <target> qid:<qid> <feature>:<value> <feature>:<value> ... <feature>:<value> # <info>
    target = 1 if row['tconst'] in relevant else 0
    features = extract_features(row)

    return f"{target} qid:1{features} # {row['tconst']}\n"


In [125]:
query = queries[0]
relevant = list(map(lambda el: el.strip(), open("../qrels_files/christmas_movies.txt").readlines()))

results = requests.get(query).json()["response"]["docs"]
df = pd.DataFrame(results)

lines = df.apply(lambda row: convert_to_dat(row, relevant), axis=1)
with open("../ltr/data/train.dat", 'w') as file:
    file.writelines(lines)